<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import pandas as pd
# Display all the columns
pd.set_option('display.max_columns', None)
path_input='./../data/data_raw/'
path_output='./../data/data_created/' 

* input
    * sendInBlue.csv
    * Req_HI_Lab_Users.csv
* output
    * 20200728_simon_fahmi_mailopnening.csv
    * 20200728_simon_fahmi_reasonsopnening.csv
    

In [ ]:
df = pd.read_csv(path_output+"sendInBlue.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df["st_text"].value_counts()

In [ ]:
df[df["st_text"] == 'Envoyé'].shape[0]

In [ ]:
deliverability_rate = round(((df[df["st_text"] == 'Délivré'].shape[0]/df[df["st_text"] == 'Envoyé'].shape[0])*100),2)
print(f"deliverability rate = {deliverability_rate}%")

In [ ]:
open_rate = round(((df[df["st_text"] == 'Ouvert'].shape[0]/df[df["st_text"] == 'Délivré'].shape[0])*100),2)
print(f"open rate = {open_rate}%")

In [ ]:
df["PretID"].nunique()

In [ ]:
df[df["PretID"] == 'A32433']

In [ ]:
df["Status"].value_counts()

In [ ]:
df_Req_HI_Lab_Users = pd.read_csv(path_output+'Req_HI_Lab_Users.csv')

In [ ]:
df_Req_HI_Lab_Users.sample(5)

In [ ]:
df_Req_HI_Lab_Users["ID_rq"].nunique()

In [ ]:
df_Req_HI_Lab_Users_2 = df_Req_HI_Lab_Users[["ID_rq", "Name_hi", "Name_lab"]]

In [ ]:
df_Req_HI_Lab_Users_2.head()

In [ ]:
df_Req_HI_Lab_Users_2.dtypes

In [ ]:
df_mail2 = df.copy()

In [ ]:
df_mail2['PretID'] = df_mail2['PretID'].str[1:]

In [ ]:
df_mail2.dtypes

In [ ]:
df_mail2['PretID'].isna().sum()

In [ ]:
df_mail2[df_mail2["PretID"] == 'nknow']

In [ ]:
mask_drop_nkow = df_mail2["PretID"] == 'nknow'

df_mail3 = df_mail2.loc[-mask_drop_nkow,:]
df_mail3.shape

In [ ]:
df_mail3['PretID'] =  df_mail3['PretID'].astype('int64')

In [ ]:
print(df_mail3.dtypes, df_Req_HI_Lab_Users_2.dtypes, end=" ")

In [ ]:
#df_mail4 = pd.merge(df_mail3, df_Req_HI_Lab_Users_2)
df_mail4 = df_mail3.merge(df_Req_HI_Lab_Users_2, left_on='PretID', right_on='ID_rq')

In [ ]:
df_mail4.shape

In [ ]:
df_mail4.head()

In [ ]:
df_lab_answer = df_mail4.groupby(['Name_lab'])['PretID'].nunique()

In [ ]:
df_mail4.groupby(['Name_lab'])['ID_rq'].nunique().sort_values()

In [ ]:
df_lab_answer

In [ ]:
#df_lab_answer = df_mail4.groupby(['Name_lab'])['PretID'].nunique()

df_mail4.groupby(["Name_lab", "PretID"])["PretID"].nunique()

In [ ]:
df_mail4.groupby(["Name_lab", "PretID"])["st_text"].apply(lambda ser: ser.str.contains("Ouvert").nunique())

#df_mail4.groupby(["Name_lab", "PretID"])["st_text"].apply(lambda ser: ser.str.contains("Ouvert"))

In [ ]:
df_mail_ouvert = df_mail4.groupby(["Name_lab", "PretID"])["st_text"].apply(lambda ser: ser.str.contains("Ouvert").nunique()).to_frame()

In [ ]:
df_mail_ouvert["st_text"].value_counts()

In [ ]:
#création d'une colonne booléen

df_mail4['bool_open'] = 0
cond_open = df_mail4['st_text'].str.contains("Ouvert")
cond_open2 = df_mail4['st_text'].str.contains("1ère ouverture")
df_mail4.loc[cond_open, 'bool_open'] = 1
df_mail4.loc[cond_open2, 'bool_open'] = 1
df_mail4.sample(20)

In [ ]:
df_mail4.groupby(["Name_lab", "PretID"])["bool_open"].sum()

In [ ]:
dfmail_final = df_mail4.groupby(["Name_lab", "PretID"])["bool_open"].sum().to_frame()

In [ ]:
dfmail_final.head()

In [ ]:
#1 sur la colonne booléen

cond_1 = dfmail_final['bool_open'] > 0
dfmail_final.loc[cond_1, 'bool_open'] = 1
dfmail_final.sample(20)

In [ ]:
df_openfin = dfmail_final.groupby(["Name_lab"])["bool_open"].sum().to_frame()

In [ ]:
df_totfin = dfmail_final.groupby(["Name_lab"])["bool_open"].count().to_frame()

In [ ]:
df_openfin.head()

In [ ]:
df_totfin.head()

In [ ]:
df_finalfinal = pd.merge(df_openfin, df_totfin, left_index=True, right_index=True)

In [ ]:
df_finalfinal.head()

In [ ]:
df_finalfinal["open_rate"] = round((df_finalfinal["bool_open_x"]/df_finalfinal["bool_open_y"])*100, 2)

In [ ]:
df_finalfinal.tail()

In [ ]:
df_finalfinal["open_rate"].sort_values()

In [ ]:
df_finalfinal.to_csv(path_output+"simon_fahmi_mailopnening.csv")

In [ ]:
df_mail4.shape

In [ ]:
df_mail4.groupby(["Status", "ID_rq"])["bool_open"].sum()

In [ ]:
df_reasons_1 = df_mail4.groupby(["Status", "ID_rq"])["bool_open"].sum().to_frame()

In [ ]:
df_reasons_1

In [ ]:
cond_2 = df_reasons_1['bool_open'] > 0
df_reasons_1.loc[cond_2, 'bool_open'] = 1
df_reasons_1

In [ ]:
df_openreason = df_reasons_1.groupby(["Status"])["bool_open"].sum().to_frame()
df_totreason = df_reasons_1.groupby(["Status"])["bool_open"].count().to_frame()


In [ ]:
df_openreason

In [ ]:
df_totreason

In [ ]:
df_reasonfinal = pd.merge(df_openreason, df_totreason, left_index=True, right_index=True)

In [ ]:
df_reasonfinal["open_rate"] = round((df_reasonfinal["bool_open_x"]/df_reasonfinal["bool_open_y"])*100, 2)

In [ ]:
df_reasonfinal

In [ ]:
df_mail4["Status"].isnull().sum()

In [ ]:
df_reasonfinal.to_csv(path_output+"simon_fahmi_reasonsopnening.csv")